## 1. Install packages, GPU and Seed

In [1]:
# Standard libraries
import h5py
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors
import numpy as np
import pandas as pd
import random
import os
import pickle
import gc
from collections import Counter

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# Sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

# Custom
from cust_functions.processing_helper import *
from cust_functions.models import *
from cust_functions.training_helper import *
from cust_functions.datavis_helper import *

In [2]:
print("Torch CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
print("Current GPU:", torch.cuda.current_device())
print("GPU Name:", torch.cuda.get_device_name(torch.cuda.current_device()))

Torch CUDA available: True
Number of GPUs: 2
Current GPU: 0
GPU Name: NVIDIA GeForce GTX 1080 Ti


In [3]:
"""
Define seed
"""

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = False

## 2. Validation set (optional, data already precomputed)

In [15]:
directory = "/home"
base_path = f"{directory}/pf/pfstud/mlarriere/aligned_data/"
dw_path = f"{directory}/pf/pfstud/mlarriere/DW_files/DW_timeslices/"

tile_lengths = {'tile0': 36, 'tile1': 26, 'tile3': 16, 'tile4': 22}
tiles = [0, 1, 3, 4]

# create files for all h5 image tiles
tile0 = [f"{base_path}tile0_aligned_{timestep}.h5" for timestep in range(0, tile_lengths['tile0'])]
tile1 = [f"{base_path}tile1_aligned_{timestep}.h5" for timestep in range(0, tile_lengths['tile1'])]
tile3 = [f"{base_path}tile3_aligned_{timestep}.h5" for timestep in range(0, tile_lengths['tile3'])]
tile4 = [f"{base_path}tile4_aligned_{timestep}.h5" for timestep in range(0, tile_lengths['tile4'])]
h5_files = tile0 + tile1 + tile3 + tile4

# create files for all h5 land cover tiles
dw_tile0 = [f"{dw_path}scaled_cloudy_tile_0_dw_{timestep}.h5" for timestep in range(0, tile_lengths['tile0'])]
dw_tile1 = [f"{dw_path}scaled_cloudy_tile_1_dw_{timestep}.h5" for timestep in range(0, tile_lengths['tile1'])]
dw_tile3 = [f"{dw_path}scaled_cloudy_tile_3_dw_{timestep}.h5" for timestep in range(0, tile_lengths['tile3'])]
dw_tile4 = [f"{dw_path}scaled_cloudy_tile_4_dw_{timestep}.h5" for timestep in range(0, tile_lengths['tile4'])]
dw_files_scaled_cloudy = dw_tile0 + dw_tile1 + dw_tile3 + dw_tile4


In [25]:
# 256 x 256 patches with 3 time slices
file_names = {'CLD': 'CLD', 'RGB': 'RGB', 'NIR': 'NIR'}
dw_file_name = 'DW_cloudy'
dataset_full = H5Dataset_early(h5_files, dw_files_scaled_cloudy, file_name = file_names, dw_file_name=dw_file_name, patch_size=(256, 256), cloud_threshold = 1.0, 
                         calculate_time_slice_indices=False, time_file_name='time_slice_indices_precomputed_256_early3.pkl', timesteps=3)

In [16]:
# 256 x 256 patches with 5 time slices
file_names = {'CLD': 'CLD', 'RGB': 'RGB', 'NIR': 'NIR'}
dw_file_name = 'DW_cloudy'
dataset_full5 = H5Dataset_early(h5_files, dw_files_scaled_cloudy, file_name = file_names, dw_file_name=dw_file_name, patch_size=(256, 256), cloud_threshold = 1.0, 
                         calculate_time_slice_indices=False, time_file_name='time_slice_indices_precomputed_256_early5.pkl', timesteps=5)

In [26]:
# load indices from file for 256 x 256 patches with 3 time slices
with open('val_indices_256.pkl', 'rb') as f:
    val_indices_256 = pickle.load(f)

val_dataset_256_early3 = torch.utils.data.Subset(dataset_full, val_indices_256)

11630


In [17]:
# load indices from file for 256 x 256 patches with 5 time slices
with open('val_indices_256.pkl', 'rb') as f:
    val_indices_256 = pickle.load(f)
    print(len(val_indices_256))

val_dataset_256_early5 = torch.utils.data.Subset(dataset_full5, val_indices_256)
save_dir = '/home/pf/pfstud/mlarriere/preprocessed_val_data_256_early5'
save_preprocessed_data(val_dataset_256_early5, save_dir)

11630


## 3. Model pipelines

In [4]:
### Load val datasets ###
with open('precomputed_indices/val_indices_256.pkl', 'rb') as f:
    val_indices_256 = pickle.load(f)
    
preprocessed_val_dataset_256_early3 = PreprocessedValDataset('/home/pf/pfstud/mlarriere/preprocessed_val_data_256_early3')
preprocessed_val_dataset_256_early5 = PreprocessedValDataset('/home/pf/pfstud/mlarriere/preprocessed_val_data_256_early5')

#### 3.1. 3 Time Points

In [6]:
directory = "/home"
# directory = "P:"
base_path = f"{directory}/pf/pfstud/mlarriere/aligned_data/"
dw_path = f"{directory}/pf/pfstud/mlarriere/DW_files/DW_timeslices/"

# File names with time steps
tile_lengths = {'tile0': 36, 'tile1': 26, 'tile3': 16, 'tile4': 22}
tiles = [0, 1, 3, 4]


# create files for all h5 image tiles
tile0 = [f"{base_path}tile0_aligned_{timestep}.h5" for timestep in range(0, tile_lengths['tile0'])]
tile1 = [f"{base_path}tile1_aligned_{timestep}.h5" for timestep in range(0, tile_lengths['tile1'])]
tile3 = [f"{base_path}tile3_aligned_{timestep}.h5" for timestep in range(0, tile_lengths['tile3'])]
tile4 = [f"{base_path}tile4_aligned_{timestep}.h5" for timestep in range(0, tile_lengths['tile4'])]
h5_files = tile0 + tile1 + tile3 + tile4

# create files for all h5 land cover tiles
dw_tile0 = [f"{dw_path}scaled_cloudy_tile_0_dw_{timestep}.h5" for timestep in range(0, tile_lengths['tile0'])]
dw_tile1 = [f"{dw_path}scaled_cloudy_tile_1_dw_{timestep}.h5" for timestep in range(0, tile_lengths['tile1'])]
dw_tile3 = [f"{dw_path}scaled_cloudy_tile_3_dw_{timestep}.h5" for timestep in range(0, tile_lengths['tile3'])]
dw_tile4 = [f"{dw_path}scaled_cloudy_tile_4_dw_{timestep}.h5" for timestep in range(0, tile_lengths['tile4'])]
dw_files_scaled_cloudy = dw_tile0 + dw_tile1 + dw_tile3 + dw_tile4

file_names = {'CLD': 'CLD', 'RGB': 'RGB', 'NIR': 'NIR'}
dw_file_name = 'DW_cloudy'


In [ ]:
cld_threshold = 1.0
dataset_fusion_early3 = H5Dataset_early(h5_files, dw_files_scaled_cloudy, file_name = file_names, dw_file_name=dw_file_name, patch_size=(256, 256), cloud_threshold = 1.0, 
                         calculate_time_slice_indices=False, time_file_name='time_slice_indices_precomputed_256_early3.pkl')
dataset_fusion_early3 = torch.utils.data.Subset(dataset_fusion_early3, list(set(range(len(dataset_fusion_early3))) - set(val_dataset_256_early3.indices)))


In [32]:
# Define model parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

NUM_CHANNELS = 12 # input dimension: 4 channels (R, G, B, NIR)
NUM_CLASSES = 11
class_labels = [0, 1, 2, 3, 4, 5, 6, 8, 7, 9, 10]  # 0-10
exlude_classes = [0, 8, 10] #8: snow & ice, 10: clouds
reduced_classes = [1, 2, 3, 4, 5, 6, 7, 9] #1: trees, 2: grass, 3: flooded veg., 4: crops, 5: shrub & scrub, 6: built-up, 7: bare, 9: water

num_epochs = 40
lr = 0.001 # 0.001
weight_decay = 1e-5
batch_size_train = 32
batch_size_val = 32

train_dataloader = DataLoader(dataset_fusion_early3, batch_size=batch_size_train, shuffle=False, num_workers=8)
val_dataloader = DataLoader(preprocessed_val_dataset_256_early3, batch_size=batch_size_val, shuffle=False, num_workers=8)
model = Unet_DW(NUM_CHANNELS, NUM_CLASSES)
model.to(device)
model = nn.DataParallel(model)
weights = torch.tensor([0.05, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.05, 1.0, 1.0]).to(device) 
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.25, patience=5)

# Initiate epoch metrics
train_results = {
    "train_loss": [],
}

val_results = {
    "val_loss": [],
    "best_val_accuracy": 0.0,
    "best_val_f1_score": 0.0,
    "best_val_recall": 0.0,
    "best_val_precision": 0.0,
}

# store best model of reduced classes
best_f1_scores_per_class = {f'class{i}': -1 for i in reduced_classes}

best_val_loss = np.inf
best_model = None
best_epoch = 0
patience = 10


cuda


In [ ]:
# Train model
for epoch in range(num_epochs):   

    # train and validate model
    running_loss = train_model(model, optimizer, criterion, train_dataloader, device, NUM_CLASSES)
    running_val_loss = validate_model_simple(model, criterion, val_dataloader, device)

     # Early stopping
    if running_val_loss < best_val_loss:
        best_val_loss = running_val_loss
        best_model = model.state_dict().copy()
        best_epoch = epoch
        patience = 10
    else:
        patience -= 1
        if patience == 0:
            print("Early stopping")
            break

    # Calculate epoch metrics
    epoch_train_loss = running_loss / len(train_dataloader)
    epoch_val_loss = running_val_loss / len(val_dataloader)
    
    # Update results lists
    train_results["train_loss"].append(epoch_train_loss)
    val_results["val_loss"].append(epoch_val_loss)

    print(f"Epoch {epoch+1}: train loss {epoch_train_loss:.4f}, validation loss {epoch_val_loss:.4f}") 

    # Update learning rate
    scheduler.step(epoch_val_loss)

# Do one full validation with the best model
model.load_state_dict(best_model)
_, running_val_confusion_matrix = validate_model(model, criterion, val_dataloader, device, NUM_CLASSES, class_labels)
running_val_confusion_matrix_reduced = calculate_reduced_confusion_matrix(running_val_confusion_matrix, exlude_classes)

# Update best f1 scores per class
val_precision, val_recall, val_f1_score = calculate_metrics(running_val_confusion_matrix_reduced)

for i, f1 in enumerate(val_f1_score):
    best_f1_scores_per_class[f'class{i}'] = f1

# Calculate average metrics
val_precision_avg = val_precision.mean()
val_recall_avg = val_recall.mean()
val_f1_score_avg = val_f1_score.mean()
val_accuracy = running_val_confusion_matrix_reduced.trace() / running_val_confusion_matrix.sum()

# Update results lists
# replace last value with best value of best epoch
val_results["best_val_accuracy"] = val_accuracy
val_results["best_val_precision"] = val_precision_avg
val_results["best_val_recall"] = val_recall_avg
val_results["best_val_f1_score"] = val_f1_score_avg

# Save model
saved_f1 = np.round(val_results["best_val_f1_score"], 2)
PATH = f"EarlyFusioCNN3_bestf1_{saved_f1}.pth"
torch.save(best_model, PATH)

# Plot results
plot_loss(train_results, val_results)

#### 3.2 5 Time Points

In [20]:
cld_threshold = 1.0
file_names = {'CLD': 'CLD', 'RGB': 'RGB', 'NIR': 'NIR'}
dw_file_name = 'DW_cloudy'
dataset_fusion_early5 = H5Dataset_early(h5_files, dw_files_scaled_cloudy, file_name = file_names, dw_file_name=dw_file_name, patch_size=(256, 256), cloud_threshold = 1.0, 
                         calculate_time_slice_indices=False, time_file_name='time_slice_indices_precomputed_256_early5.pkl', timesteps=5)
dataset_fusion_early5 = torch.utils.data.Subset(dataset_fusion_early5, list(set(range(len(dataset_fusion_early5))) - set(val_dataset_256_early5.indices)))


In [21]:
# Define model parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

NUM_CHANNELS = 20 # input dimension: 4 channels (R, G, B, NIR)
NUM_CLASSES = 11
class_labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # 0-10
exlude_classes = [0, 8, 10] #8: snow & ice, 10: clouds
reduced_classes = [1, 2, 3, 4, 5, 6, 7, 9] #1: trees, 2: grass, 3: flooded veg., 4: crops, 5: shrub & scrub, 6: built-up, 7: bare, 9: water

num_epochs = 40
lr = 0.001 # 0.001
weight_decay = 1e-5
batch_size_train = 32
batch_size_val = 16

train_dataloader = DataLoader(dataset_fusion_early5, batch_size=batch_size_train, shuffle=False, num_workers=8)
val_dataloader = DataLoader(val_dataset_256_early5, batch_size=batch_size_val, shuffle=False, num_workers=8)
model = Unet_DW(NUM_CHANNELS, NUM_CLASSES)
model.to(device)
model = nn.DataParallel(model)
weights = torch.tensor([0.05, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.05, 1.0, 1.0]).to(device) 
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.25, patience=5)

# Initiate epoch metrics
train_results = {
    "train_loss": [],
}

val_results = {
    "val_loss": [],
    "best_val_accuracy": 0.0,
    "best_val_f1_score": 0.0,
    "best_val_recall": 0.0,
    "best_val_precision": 0.0,
}

# store best model of reduced classes
best_f1_scores_per_class = {f'class{i}': -1 for i in reduced_classes}

best_val_loss = np.inf
best_model = None
best_epoch = 0
patience = 10


cuda


In [ ]:
# Train model
for epoch in range(num_epochs):   

    # train and validate model
    running_loss = train_model(model, optimizer, criterion, train_dataloader, device, NUM_CLASSES)
    running_val_loss = validate_model_simple(model, criterion, val_dataloader, device)

     # Early stopping
    if running_val_loss < best_val_loss:
        best_val_loss = running_val_loss
        best_model = model.state_dict().copy()
        best_epoch = epoch
        patience = 10
    else:
        patience -= 1
        if patience == 0:
            print("Early stopping")
            break

    # Calculate epoch metrics
    epoch_train_loss = running_loss / len(train_dataloader)
    epoch_val_loss = running_val_loss / len(val_dataloader)
    
    # Update results lists
    train_results["train_loss"].append(epoch_train_loss)
    val_results["val_loss"].append(epoch_val_loss)

    print(f"Epoch {epoch+1}: train loss {epoch_train_loss:.4f}, validation loss {epoch_val_loss:.4f}") 

    # Update learning rate
    scheduler.step(epoch_val_loss)

# Do one full validation with the best model
model.load_state_dict(best_model)
_, running_val_confusion_matrix = validate_model(model, criterion, val_dataloader, device, NUM_CLASSES, class_labels)
running_val_confusion_matrix_reduced = calculate_reduced_confusion_matrix(running_val_confusion_matrix, exlude_classes)

# Update best f1 scores per class
val_precision, val_recall, val_f1_score = calculate_metrics(running_val_confusion_matrix_reduced)

for i, f1 in enumerate(val_f1_score):
    best_f1_scores_per_class[f'class{i}'] = f1

# Calculate average metrics
val_precision_avg = val_precision.mean()
val_recall_avg = val_recall.mean()
val_f1_score_avg = val_f1_score.mean()
val_accuracy = running_val_confusion_matrix_reduced.trace() / running_val_confusion_matrix.sum()

# Update results lists
# replace last value with best value of best epoch
val_results["best_val_accuracy"] = val_accuracy
val_results["best_val_precision"] = val_precision_avg
val_results["best_val_recall"] = val_recall_avg
val_results["best_val_f1_score"] = val_f1_score_avg

# Save model
saved_f1 = np.round(val_results["best_val_f1_score"], 2)
PATH = f"EarlyFusionCNN5_bestf1_{saved_f1}.pth"
torch.save(best_model, PATH)

# Plot results
plot_loss(train_results, val_results)

### 3.3 With Attention, 3 Time Points

In [ ]:
cld_threshold = 1.0
file_names = {'CLD': 'CLD', 'RGB': 'RGB', 'NIR': 'NIR'}
dw_file_name = 'DW_cloudy'
dataset_fusion_early3 = H5Dataset_early(h5_files, dw_files_scaled_cloudy, file_name = file_names, dw_file_name=dw_file_name, patch_size=(256, 256), cloud_threshold = 1.0, 
                         calculate_time_slice_indices=False, time_file_name='time_slice_indices_precomputed_256_early3.pkl')
# exlude validation dataset from baseline dataset
dataset_fusion_early3 = torch.utils.data.Subset(dataset_fusion_early3, list(set(range(len(dataset_fusion_early3))) - set(val_dataset_256_early3.indices)))


In [ ]:
# Define model parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

NUM_CHANNELS = 12 # input dimension: 4 channels (R, G, B, NIR)
NUM_CLASSES = 11
class_labels = [0, 1, 2, 3, 4, 5, 6, 8, 7, 9, 10]  # 0-10
exlude_classes = [0, 8, 10] #8: snow & ice, 10: clouds
reduced_classes = [1, 2, 3, 4, 5, 6, 7, 9] #1: trees, 2: grass, 3: flooded veg., 4: crops, 5: shrub & scrub, 6: built-up, 7: bare, 9: water

num_epochs = 40
lr = 0.001 # 0.001
weight_decay = 1e-5
batch_size_train = 32
batch_size_val = 32

train_dataloader = DataLoader(dataset_fusion_early3, batch_size=batch_size_train, shuffle=False, num_workers=8)
val_dataloader = DataLoader(preprocessed_val_dataset_256_early3, batch_size=batch_size_val, shuffle=False, num_workers=8)
model = Unet_DW_attention(NUM_CHANNELS, NUM_CLASSES)
model.to(device)
model = nn.DataParallel(model)
weights = torch.tensor([0.05, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.05, 1.0, 1.0]).to(device) 
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.25, patience=5)

# Initiate epoch metrics
train_results = {
    "train_loss": [],
}

val_results = {
    "val_loss": [],
    "best_val_accuracy": 0.0,
    "best_val_f1_score": 0.0,
    "best_val_recall": 0.0,
    "best_val_precision": 0.0,
}

# store best model of reduced classes
best_f1_scores_per_class = {f'class{i}': -1 for i in reduced_classes}

best_val_loss = np.inf
best_model = None
best_epoch = 0
patience = 10

In [ ]:
# Train model
for epoch in range(num_epochs):   

    # train and validate model
    running_loss = train_model(model, optimizer, criterion, train_dataloader, device, NUM_CLASSES)
    running_val_loss = validate_model_simple(model, criterion, val_dataloader, device)

     # Early stopping
    if running_val_loss < best_val_loss:
        best_val_loss = running_val_loss
        best_model = model.state_dict().copy()
        best_epoch = epoch
        patience = 10
    else:
        patience -= 1
        if patience == 0:
            print("Early stopping")
            break

    # Calculate epoch metrics
    epoch_train_loss = running_loss / len(train_dataloader)
    epoch_val_loss = running_val_loss / len(val_dataloader)
    
    # Update results lists
    train_results["train_loss"].append(epoch_train_loss)
    val_results["val_loss"].append(epoch_val_loss)

    print(f"Epoch {epoch+1}: train loss {epoch_train_loss:.4f}, validation loss {epoch_val_loss:.4f}") 

    # Update learning rate
    scheduler.step(epoch_val_loss)

# Do one full validation with the best model
model.load_state_dict(best_model)
_, running_val_confusion_matrix = validate_model(model, criterion, val_dataloader, device, NUM_CLASSES, class_labels)
running_val_confusion_matrix_reduced = calculate_reduced_confusion_matrix(running_val_confusion_matrix, exlude_classes)

# Update best f1 scores per class
val_precision, val_recall, val_f1_score = calculate_metrics(running_val_confusion_matrix_reduced)

for i, f1 in enumerate(val_f1_score):
    best_f1_scores_per_class[f'class{i}'] = f1

# Calculate average metrics
val_precision_avg = val_precision.mean()
val_recall_avg = val_recall.mean()
val_f1_score_avg = val_f1_score.mean()
val_accuracy = running_val_confusion_matrix_reduced.trace() / running_val_confusion_matrix.sum()

# Update results lists
# replace last value with best value of best epoch
val_results["best_val_accuracy"] = val_accuracy
val_results["best_val_precision"] = val_precision_avg
val_results["best_val_recall"] = val_recall_avg
val_results["best_val_f1_score"] = val_f1_score_avg

# Save model
saved_f1 = np.round(val_results["best_val_f1_score"], 2)
PATH = f"EarlyFusioCNN3_bestf1_{saved_f1}.pth"
torch.save(best_model, PATH)

# Plot results
plot_loss(train_results, val_results)